In [1]:
import sys
import torch as t
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
import numpy as np
import einops
from jaxtyping import Int, Float
import functools
from tqdm import tqdm
from IPython.display import display
from transformer_lens.hook_points import HookPoint
from transformer_lens import (
    utils,
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)
import circuitsvis as cv

from optim_hunter.plotly_utils import imshow, hist, plot_comp_scores, plot_logit_attribution, plot_loss_difference, line
from optim_hunter.utils import prepare_prompt, slice_dataset
from optim_hunter.sklearn_regressors import linear_regression, knn_regression, random_forest, baseline_average, baseline_last, baseline_random
from optim_hunter.datasets import get_dataset_friedman_2
import logging

# Configure logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Saves computation time, since we don't need it for the contents of this notebook
t.set_grad_enabled(False)

#device = t.device("cuda:0,1" if t.cuda.is_available() else "cpu")
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
# device = t.device("cpu")

MAIN = __name__ == "__main__"

In [2]:
# Load directly from model path https://github.com/TransformerLensOrg/TransformerLens/issues/691
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

MODEL_TYPE = "meta-llama/Meta-Llama-3-8B-Instruct"
MODEL_PATH = "/home/freiza/optim_hunter/.models/Llama-3.1-8B-Instruct/"

if MODEL_PATH:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    hf_model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True,
                                                     #quantization_config=BitsAndBytesConfig(load_in_4bit=True), 
                                                     #torch_dtype = t.float32, 
                                                     #device_map = "cuda:0"
                                                     )

    tokenizer.padding_side = 'left'
    tokenizer.pad_token = tokenizer.eos_token

    model = HookedTransformer.from_pretrained(
        MODEL_TYPE,
        hf_model=hf_model,
        device="cuda",
        n_devices=2,
        fold_ln=True,
        # fold_value_biases=False,
        center_writing_weights=True,
        # refactor_factored_attn_matrices=True,
        center_unembed=True,
        # dtype=t.bfloat16,
        dtype=t.float16,
        default_padding_side='left',
        tokenizer=tokenizer
    )

    #model = model.to("cuda" if t.cuda.is_available() else "cpu")
    #model.generate("The capital of Germany is", max_new_tokens=20, temperature=0)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded pretrained model meta-llama/Meta-Llama-3-8B-Instruct into HookedTransformer


In [3]:
from optim_hunter.model_utils import get_numerical_tokens

numerical_tokens = get_numerical_tokens(model)

print("Found numerical tokens:", numerical_tokens)
print("Number of numerical tokens:", len(numerical_tokens))


# Check if digit 0 is in numerical tokens
has_zero = '0' in numerical_tokens
print("Zero token present:", has_zero)
if has_zero:
    print("Token ID for zero:", numerical_tokens['29'])


Found numerical tokens: {'667': 19774, '62': 5538, '754': 23952, '979': 25476, '078': 24850, '747': 23619, '94': 6281, '327': 13817, '744': 23800, '660': 19274, '321': 14423, '451': 20360, '781': 21893, '285': 15935, '47': 2618, '836': 25192, '550': 13506, '006': 11030, '805': 21032, '654': 21969, '44': 2096, '624': 23000, '408': 18058, '528': 21458, '300': 3101, '024': 19592, '848': 24951, '78': 2495, '221': 12425, '888': 12251, '376': 18322, '652': 23181, '877': 23873, '437': 18318, '875': 17419, '018': 16745, '431': 19852, '714': 23193, '533': 21876, '184': 10336, '947': 26511, '81': 5932, '99': 1484, '005': 8504, '155': 9992, '504': 18048, '91': 5925, '449': 21125, '443': 17147, '961': 26114, '580': 18216, '662': 24199, '743': 26260, '978': 17272, '483': 21884, '937': 21936, '357': 18520, '177': 11242, '020': 11139, '487': 22184, '512': 8358, '597': 24574, '307': 14777, '618': 21985, '471': 20617, '197': 4468, '766': 25358, '858': 23805, '067': 27309, '312': 13384, '027': 21360, '5

In [ ]:
seq_len = 5
x_train, y_train, x_test, y_test = get_dataset_friedman_2(random_state=11)
# x_train, y_train, x_test, y_test =  slice_dataset(x_train, y_train, x_test, y_test, seq_len)
linear_regression_prediction = linear_regression(x_train, x_test, y_train, y_test)['y_predict']
knn_regression_prediction = knn_regression(x_train, x_test, y_train, y_test)['y_predict']
random_forest_prediction = random_forest(x_train, x_test, y_train, y_test)['y_predict']
baseline_avg_prediction = baseline_average(x_train, x_test, y_train, y_test)['y_predict']
baseline_last_prediction = baseline_last(x_train, x_test, y_train, y_test)['y_predict']
baseline_random_prediction = baseline_random(x_train, x_test, y_train, y_test)['y_predict']
gold = y_test.values[0]

print("Linear Regression Prediction:", linear_regression_prediction[0])
print("Random Forest Prediction:", random_forest_prediction[0])
print("KNN Regression Prediction:", knn_regression_prediction[0])
print("Baseline Average Prediction:", baseline_avg_prediction[0])
print("Baseline Last Prediction:", baseline_last_prediction[0]) 
print("Baseline Random Prediction:", baseline_random_prediction[0])
print("Gold:", gold)

prompt = prepare_prompt(x_train, y_train, x_test)
prompt = prompt + ""
# example_answer = f"{baseline_last_prediction[0]}"
# example_answer = f"{random_forest_prediction[0]}"
example_answer = f"{gold}"
utils.test_prompt(prompt, example_answer, model, prepend_bos=True)

In [4]:
def create_comparison_data(model, dataset_func, regressors, random_state=1, seq_len=None):
    """
    Creates a structured comparison dataset for analyzing different regression models against gold values.
    
    Args:
        model (HookedTransformer): The transformer model used for tokenization
        dataset_func (callable): Function that returns (x_train, y_train, x_test, y_test)
        regressors (list): List of regression functions to compare
        random_state (int, optional): Random seed for dataset generation. Defaults to 11
    
    Returns:
        dict: A structured dictionary containing:
        {
            'dataset_name': str,  # Name of the dataset function
            'prompt': str,        # Generated prompt text for the model
            'predictions': {      # Dictionary of predictions from each model
                'gold': float,    # True value
                'model_name1': float,  # Prediction from first model
                'model_name2': float,  # Prediction from second model
                ...
            },
            'comparison_names': [  # List of comparison descriptors
                'model1 vs model2',
                'model1 vs model3',
                ...
            ],
            'token_pairs': tensor  # Shape: [num_comparisons, 1, 2]
                                  # Each pair contains the first tokens of two predictions
                                  # being compared
        }
    
    Note:
        - The function generates unique combinations (not permutations) of comparisons
        - Only the first token of each prediction is stored in token_pairs
        - All possible combinations between gold and regressors are included
        - Token pairs maintain the order specified in comparison_names
    """
    # Get dataset
    x_train, y_train, x_test, y_test = dataset_func(random_state=random_state)
    if seq_len:
        x_train, y_train, x_test, y_test = slice_dataset(x_train, y_train, x_test, y_test, seq_len)
    
    # Get prompt
    prompt = prepare_prompt(x_train, y_train, x_test)
    
    # Get gold value
    gold = y_test.values[0]
    
    # Get predictions from each regressor
    predictions = {}
    predictions['gold'] = gold
    if regressors:
        for regressor in regressors:
            result = regressor(x_train, x_test, y_train, y_test)
            predictions[result['model_name']] = result['y_predict'][0]
    
    # Create comparison names and token pairs
    comparison_names = []
    token_pairs = []
    
    # Create list of all predictors (including gold)
    if regressors:
        all_predictors = ['gold'] + [reg(x_train, x_test, y_train, y_test)['model_name'] for reg in regressors]
    else:
        all_predictors = ['gold']

    # Generate unique combinations (not permutations)
    for i, pred1 in enumerate(all_predictors):
        for j, pred2 in enumerate(all_predictors[i+1:], i+1):  # Start from i+1 to avoid duplicates
            comparison_name = f"{pred1} vs {pred2}"
            comparison_names.append(comparison_name)
            
            # Tokenize each prediction separately and get their first tokens
            tokens1 = model.to_tokens(str(predictions[pred1]), prepend_bos=False)[0, 0]  # First token of first prediction
            tokens2 = model.to_tokens(str(predictions[pred2]), prepend_bos=False)[0, 0]  # First token of second prediction
            
            # Combine the first tokens into a pair
            first_tokens = t.tensor([tokens1, tokens2], device=tokens1.device).unsqueeze(0)  # Shape: [1, 2]
            token_pairs.append(first_tokens)

    # Verification Step: Ensure that each comparison_name matches the corresponding token_pair
    for idx, (comp_name, token_pair) in enumerate(zip(comparison_names, token_pairs)):
        pred1_name, pred2_name = comp_name.split(' vs ')
        pred1_value = predictions[pred1_name]
        pred2_value = predictions[pred2_name]
        
        # Tokenize the actual prediction values
        actual_tokens1 = model.to_tokens(str(pred1_value), prepend_bos=False)[0, 0].item()
        actual_tokens2 = model.to_tokens(str(pred2_value), prepend_bos=False)[0, 0].item()
        
        # Extract tokens from token_pair
        token1, token2 = token_pair.squeeze(0).tolist()
        
        # Assert that tokens match
        assert token1 == actual_tokens1, f"Mismatch in token1 for comparison '{comp_name}' at index {idx}"
        assert token2 == actual_tokens2, f"Mismatch in token2 for comparison '{comp_name}' at index {idx}"
    
    
    return {
        'dataset_name': dataset_func.__name__,
        'prompt': prompt,
        'predictions': predictions,
        'comparison_names': comparison_names,
        'token_pairs': t.stack(token_pairs),  # Shape: [num_comparisons, 1, 2]
    }

# # Create the data store
# datasets = [ get_dataset_friedman_2 ]
# regressors = [ linear_regression, knn_regression, random_forest, baseline_average, baseline_last, baseline_random ]
# data_store = {}
# for dataset_func in datasets:
#     data_store[dataset_func.__name__] = create_comparison_data(model, dataset_func, regressors)
# # Print out the token pairs and comparison names
# for dataset_name, dataset_info in data_store.items():
#     print(f"\nDataset: {dataset_name}")
#     print("Token pairs:")
#     print(dataset_info['token_pairs'])
#     print("\nComparison names:")
#     print(dataset_info['comparison_names'])

# # Get the first token pair from the first dataset
# first_dataset_name = next(iter(data_store))
# first_token_pair = data_store[first_dataset_name]['token_pairs'][0]  # Shape: [1, 2]
# print("\nFirst token pair:")
# print(first_token_pair)

In [5]:
regressors = [ linear_regression, knn_regression, random_forest, baseline_average, baseline_last, baseline_random ]

def generate_linreg_tokens(
    model: HookedTransformer,
    dataset,
    seq_len = 5,
    batch: int = 1
) -> Int[Tensor, "batch full_seq_len"]:
    '''
    Generates a sequence of linear regression ICL tokens

    Outputs are:
        linreg_tokens: [batch, 1+linreg]
    '''
    prefix = (t.ones(batch, 1) * model.tokenizer.bos_token_id).long().to(device)
    zero_token = model.to_tokens('0', truncate=True)[0][-1]
    
    # Create list to store tokens for each batch
    batch_tokens = []
    data_store = []

    dataset_func = get_dataset_friedman_2
    
    # Generate tokens for each batch with different random seeds
    for i in range(batch):
        data = create_comparison_data(model, dataset_func, regressors, random_state=i, seq_len=seq_len)
        tokens = model.to_tokens(data['prompt'], truncate=True)
        batch_tokens.append(tokens[0])
        data_store.append(data)
    
    # Find the longest sequence length
    max_len = max(len(tokens) for tokens in batch_tokens)
    
    # Pad shorter sequences with token 0 at position -4
    for i in range(len(batch_tokens)):
        while len(batch_tokens[i]) < max_len:
            # Insert 0 at position -4 from the end
            print(f"Found mismatch in token length for batch {i}!\nLargest length: {max_len}\nBatch {i} length: {len(batch_tokens[i])}\nApplying padding...")
            print(f"\nBefore Zero Token Padding:\n#####\n{model.to_string(batch_tokens[i][-50:])}\n#####")
            batch_tokens[i] = t.cat([
                batch_tokens[i][:len(batch_tokens[i])-3],  
                zero_token.unsqueeze(0), # Add unsqueeze to make zero_token 1-dimensional
                batch_tokens[i][len(batch_tokens[i])-3:]
            ])
            print(f"\nAfter Zero Token Padding:\n#####\n{model.to_string(batch_tokens[i][-50:])}\n#####")

    
    # Stack all batches together 
    linreg_tokens = t.stack(batch_tokens).to(device)
    
    # Add prefix to each batch
    linreg_tokens = t.cat([prefix, linreg_tokens], dim=-1).to(device)
    return linreg_tokens, data_store

def run_and_cache_model_linreg_tokens(model: HookedTransformer, seq_len: int, batch: int = 1) -> tuple[Tensor, Tensor, ActivationCache]:
    '''
    Generates a sequence of linear regression ICL tokens, and runs the model on it, returning (tokens, logits, cache)

    Should use the `generate_linreg_tokens` function above

    Outputs are:
        linreg_tokens: [batch, 1+linreg]
        linreg_logits: [batch, 1+linreg, d_vocab]
        linreg_cache: The cache of the model run on linreg_tokens
    '''
    linreg_tokens, linreg_data_store = generate_linreg_tokens(model, get_dataset_friedman_2, seq_len, batch)
    linreg_logits, linreg_cache = model.run_with_cache(linreg_tokens)
    return linreg_tokens, linreg_logits, linreg_cache, linreg_data_store

In [15]:
def generate_and_compare_predictions(model, dataset_func, regressors, num_samples=5, seq_len=None):
    """
    Generate model predictions and compare against regression baselines using MSE across multiple prompts
    
    Args:
        model (HookedTransformer): The transformer model
        dataset_func (callable): Function that returns dataset splits
        regressors (list): List of regression functions to compare
        num_samples (int): Number of different prompts to generate and test
        seq_len (int, optional): Length to slice dataset
        
    Returns:
        dict: MSE scores and predictions for each sample
    """
    all_results = []
    
    # Generate multiple samples
    for i in range(num_samples):
        # Get data and predictions using create_comparison_data
        data = create_comparison_data(model, dataset_func, regressors, random_state=i, seq_len=seq_len)
        
        # Get the prompt
        prompt = data['prompt']
        
        # Generate model prediction
        pred_text = model.generate(prompt, max_new_tokens=4, temperature=0)
        # No need to convert to string since generate() returns string directly
        
        # Extract the numeric prediction from the generated text
        try:
            # Clean the prediction text - remove the prompt and keep only the generated part
            # This assumes the model's output follows the prompt
            generated_part = pred_text.replace(prompt, '').strip()
            model_pred = float(generated_part)
        except ValueError:
            print(f"Warning: Could not parse model prediction for sample {i}: {pred_text}")
            model_pred = None
            
        # Get gold value and regression predictions
        sample_results = {
            'sample_id': i,
            'predictions': {
                'llama': model_pred,  # Changed 'model' to 'llama' for clarity
                'gold': data['predictions']['gold'],
            },
            'mse_scores': {}
        }
        
        # Add predictions from all regressors
        for reg_name, pred_value in data['predictions'].items():
            if reg_name != 'gold':
                sample_results['predictions'][reg_name] = pred_value
        
        # Calculate MSE scores for all predictions including the model's
        gold = sample_results['predictions']['gold']
        for method, pred in sample_results['predictions'].items():
            if method != 'gold' and pred is not None:
                sample_results['mse_scores'][method] = (pred - gold) ** 2
                
        all_results.append(sample_results)
    
    # Calculate average MSE across all samples
    avg_mse = {method: [] for method in all_results[0]['mse_scores'].keys()}
    for result in all_results:
        for method, mse in result['mse_scores'].items():
            avg_mse[method].append(mse)
    
    avg_mse = {method: sum(scores)/len(scores) for method, scores in avg_mse.items()}
    
    return {
        'individual_results': all_results,
        'average_mse': avg_mse
    }

def plot_comparison_results(results):
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    
    # Create figure with secondary y-axis
    fig = make_subplots(rows=2, cols=1, 
                       subplot_titles=('Individual Sample Predictions', 'Average MSE Across Samples'),
                       vertical_spacing=0.3)
    
    # Colors for different methods
    methods = list(results['individual_results'][0]['predictions'].keys())
    methods.remove('gold')
    colors = px.colors.qualitative.Set3[:len(methods)]
    color_map = dict(zip(methods, colors))
    
    # Plot individual predictions
    for method in methods:
        x_vals = []
        y_vals = []
        for sample in results['individual_results']:
            x_vals.append(f"Sample {sample['sample_id']}")
            y_vals.append(sample['predictions'][method])
        
        fig.add_trace(
            go.Scatter(
                name=method,
                x=x_vals,
                y=y_vals,
                mode='lines+markers',
                line=dict(color=color_map[method])
            ),
            row=1, col=1
        )
        
        # Add gold values
        if method == methods[0]:  # Only add gold once
            gold_vals = [sample['predictions']['gold'] for sample in results['individual_results']]
            fig.add_trace(
                go.Scatter(
                    name='Gold',
                    x=x_vals,
                    y=gold_vals,
                    mode='lines+markers',
                    line=dict(color='black', dash='dash')
                ),
                row=1, col=1
            )
    
    # Plot average MSE
    fig.add_trace(
        go.Bar(
            name='Average MSE',
            x=list(results['average_mse'].keys()),
            y=list(results['average_mse'].values()),
            marker_color=[color_map[method] for method in results['average_mse'].keys()]
        ),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        title='Model vs Regression Methods Comparison Across Multiple Samples',
        showlegend=True,
        height=1000,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Update y-axes labels
    fig.update_yaxes(title_text="Prediction Value", row=1, col=1)
    fig.update_yaxes(title_text="MSE", row=2, col=1)
    
    return fig

# Run comparison
regressors = [linear_regression, knn_regression, random_forest, 
              baseline_average, baseline_last, baseline_random]

results = generate_and_compare_predictions(
    model=model,
    dataset_func=get_dataset_friedman_2,
    regressors=regressors,
    num_samples=1,
    seq_len=None
)

# Create and display visualization
fig = plot_comparison_results(results)
fig.show()

  0%|          | 0/4 [00:00<?, ?it/s]

In [16]:
model.clear_contexts()

seq_len = None
# TODO we need to be able to run more batches but not over 
batch = 1
(linreg_tokens, linreg_logits, linreg_cache, linreg_data_store) = run_and_cache_model_linreg_tokens(model, seq_len, batch)

OutOfMemoryError: CUDA out of memory. Tried to allocate 616.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 308.94 MiB is free. Including non-PyTorch memory, this process has 23.37 GiB memory in use. Of the allocated memory 22.77 GiB is allocated by PyTorch, and 301.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [7]:
def create_logit_lens_plot(logit_lens_logit_diffs, labels, comparison_name):
    """
    Creates a lightweight, themed logit lens plot suitable for web embedding.
    
    Args:
        logit_lens_logit_diffs: Tensor of logit differences
        labels: List of layer labels
        comparison_name: Name of the comparison being plotted
    
    Returns:
        str: HTML/JavaScript code for the plot using Plotly
    """
    import plotly.graph_objects as go
    
    # Convert tensor to list
    logit_diffs = logit_lens_logit_diffs.tolist()
    
    # Create the trace
    trace = go.Scatter(
        x=list(range(len(logit_diffs))),
        y=logit_diffs,
        mode='lines+markers',
        line=dict(
            color='#9ec5fe',  # Matches website theme
            width=2
        ),
        marker=dict(
            size=6,
            color='#9ec5fe',
            line=dict(
                color='#ffffff',
                width=1
            )
        ),
        hovertemplate='Layer: %{x}<br>Logit Diff: %{y:.3f}<extra></extra>'
    )

    # Create the layout
    layout = go.Layout(
        template='plotly_dark',  # Dark theme to match website
        plot_bgcolor='rgba(26,26,26,0)',  # Transparent background
        paper_bgcolor='rgba(26,26,26,0)',
        margin=dict(l=50, r=20, t=50, b=50),
        xaxis=dict(
            title='Layer',
            gridcolor='#444',
            ticktext=labels,
            tickvals=list(range(len(labels))),
            tickmode='array',
            showgrid=True,
            zeroline=False
        ),
        yaxis=dict(
            title='Logit Difference',
            gridcolor='#444',
            showgrid=True,
            zeroline=False
        ),
        title=dict(
            text=f'Logit Difference Across Layers<br><sub>{comparison_name}</sub>',
            font=dict(
                size=14,
                color='#e0e0e0'
            ),
            x=0.5,
            xanchor='center'
        ),
        hoverlabel=dict(
            bgcolor='#333',
            font_size=12,
            font_family="monospace"
        ),
        width=600,
        height=400
    )

    # Create figure
    fig = go.Figure(data=[trace], layout=layout)
    
    # Add light theme configuration
    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(
                    label="Light Theme",
                    method="relayout",
                    args=[{
                        "plot_bgcolor": "rgba(253,246,227,0)",
                        "paper_bgcolor": "rgba(253,246,227,0)",
                        "font.color": "#073642",
                        "xaxis.gridcolor": "#93a1a1",
                        "yaxis.gridcolor": "#93a1a1",
                        "title.font.color": "#073642"
                    }]
                ),
                dict(
                    label="Dark Theme",
                    method="relayout",
                    args=[{
                        "plot_bgcolor": "rgba(26,26,26,0)",
                        "paper_bgcolor": "rgba(26,26,26,0)",
                        "font.color": "#e0e0e0",
                        "xaxis.gridcolor": "#444",
                        "yaxis.gridcolor": "#444",
                        "title.font.color": "#e0e0e0"
                    }]
                )
            ],
            x=0.9,
            y=1.1,
            xanchor="right",
            yanchor="top",
        )]
    )

    # Generate minimal HTML
    plot_html = fig.to_html(
        full_html=False,
        include_plotlyjs='cdn',
        config={'displayModeBar': False}
    )
    
    return plot_html

# # Modify your existing loop to use the new plotting function
# for i, token_pair in enumerate(token_pairs):
#     logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, linreg_cache)
    # plot_html = create_logit_lens_plot(
    #     logit_lens_logit_diffs,
    #     labels,
    #     token_pairs_names[i]
    # )
    
    # # Save just the plot HTML
    # output_path = f"../docs/logit_lens_{token_pairs_names[i].replace(' ', '_')}.html"
    # with open(output_path, 'w') as f:
    #     f.write(plot_html)

In [23]:
model.clear_contexts()
linreg_tokens = linreg_tokens.to('cpu')
linreg_logits = linreg_logits.to('cpu')
linreg_cache = linreg_cache.to('cpu')

# Verify all datasets have the same comparison names
base_comparison_names = linreg_data_store[0]["comparison_names"]
all_match = all(dataset["comparison_names"] == base_comparison_names for dataset in linreg_data_store[1:])
assert all_match, "Mismatch in comparison names across datasets."

# Extract comparison names from the first dataset
token_pairs_names = base_comparison_names.copy()

# Extract token pairs across all datasets for each comparison
token_pairs = [
    t.stack([dataset["token_pairs"][i] for dataset in linreg_data_store])[0]
    for i in range(len(token_pairs_names))
]

logger.info(f"Number of comparisons: {len(token_pairs_names)}")
logger.info(f"Number of token_pairs: {len(token_pairs)}")

# Iterate over token pairs and generate plots
for i, token_pair in enumerate(token_pairs):
    logger.info(f"Processing comparison {i}: {token_pairs_names[i]}")
    token_pair = token_pair.to('cpu')

    # Print token pair shape for debugging
    print(f"Token pair shape: {token_pair.shape}")

    def logits_to_ave_logit_diff(
        logits: Float[Tensor, "batch seq d_vocab"],
        answer_tokens: Float[Tensor, "batch 2"] = token_pair,
        per_prompt: bool = False
    ) -> Float[Tensor, "*batch"]:
        '''
        Returns logit difference between the correct and incorrect answer.

        If per_prompt=True, return the array of differences rather than the average.
        '''
        # Extract token IDs for correct and incorrect answers
        correct = answer_tokens[:, 0]  # Correct token IDs
        incorrect = answer_tokens[:, 1]  # Incorrect token IDs

        # Extract logits for the final token in the sequence
        final_logits = logits[:, -1, :]  # Shape: (batch, d_vocab)

        # Get logits for the correct and incorrect answers
        correct_logits = final_logits[t.arange(final_logits.size(0)), correct]  # Shape: (batch,)
        incorrect_logits = final_logits[t.arange(final_logits.size(0)), incorrect]  # Shape: (batch,)

        # Calculate logit difference
        logit_diff = correct_logits - incorrect_logits  # Shape: (batch,)

        if per_prompt:
            return logit_diff  # Return per-prompt logit differences
        else:
            return logit_diff.mean()  # Return mean logit difference over the batch

    original_per_prompt_diff = logits_to_ave_logit_diff(linreg_logits, token_pair, per_prompt=True)
    logger.debug(f"Per prompt logit difference for comparison '{token_pairs_names[i]}': {original_per_prompt_diff}")
    original_average_logit_diff = logits_to_ave_logit_diff(linreg_logits, token_pair)
    logger.debug(f"Average logit difference for comparison '{token_pairs_names[i]}': {original_average_logit_diff}")

    # Retrieve final residual stream
    final_residual_stream: Float[Tensor, "batch seq d_model"] = linreg_cache["resid_post", -1]
    logger.debug(f"Final residual stream shape: {final_residual_stream.shape}")
    final_token_residual_stream: Float[Tensor, "batch d_model"] = final_residual_stream[:, -1, :]

    # Compute residual directions
    pair_residual_directions = model.tokens_to_residual_directions(token_pair.to('cpu'))  # [batch 2 d_model]
    logger.debug(f"Answer residual directions shape: {pair_residual_directions.shape}")

    correct_residual_directions, incorrect_residual_directions = pair_residual_directions.unbind(dim=1)
    logit_diff_directions = correct_residual_directions - incorrect_residual_directions  # [batch d_model]
    logger.debug(f"Logit difference directions shape: {logit_diff_directions.shape}")

    def residual_stack_to_logit_diff(
        residual_stack: Float[Tensor, "... batch d_model"],
        cache: ActivationCache,
        logit_diff_directions: Float[Tensor, "batch d_model"] = logit_diff_directions,
    ) -> Float[Tensor, "..."]:
        '''
        Gets the avg logit difference between the correct and incorrect answer for a given
        stack of components in the residual stream.
        '''
        # Apply LayerNorm scaling (to just the final sequence position)
        scaled_residual_stream = cache.apply_ln_to_stack(residual_stack, layer=-1, pos_slice=-1)

        logit_diff_directions = logit_diff_directions.to(dtype=scaled_residual_stream.dtype)

        logit_diff_directions = logit_diff_directions.to('cpu')

        logger.debug(f"Scaled residual stream shape: {scaled_residual_stream.shape}")
        logger.debug(f"Logit diff directions shape: {logit_diff_directions.shape}")

        # Projection
        batch_size = residual_stack.size(-2)
        avg_logit_diff = einops.einsum(
            scaled_residual_stream,
            logit_diff_directions,
            "... batch d_model, batch d_model -> ..."
        ) / batch_size
        return avg_logit_diff

    # Verify residual stack computation
    t.testing.assert_close(
        residual_stack_to_logit_diff(final_token_residual_stream.to(t.float32), linreg_cache.to(t.float32)),
        original_average_logit_diff.to(t.float32),
        rtol=5e-3,  # Increased tolerance
        atol=5e-3
    )

    # Accumulate residuals
    accumulated_residual, labels = linreg_cache.accumulated_resid(layer=-1, incl_mid=True, pos_slice=-1, return_labels=True)
    # accumulated_residual has shape (component, batch, d_model)

    logit_lens_logit_diffs: Float[Tensor, "component"] = residual_stack_to_logit_diff(accumulated_residual, linreg_cache)
    # Convert to half precision
    logit_lens_logit_diffs = logit_lens_logit_diffs.half()

    per_layer_residual, labels = linreg_cache.decompose_resid(layer=-1, pos_slice=-1, return_labels=True)
    per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, linreg_cache)

    # # # Generate plot
    line(
        logit_lens_logit_diffs,
        hovermode="x unified",
        title=f"Logit Difference From Accumulated Residual Stream for {token_pairs_names[i]}",
        labels={"x": "Layer", "y": "Logit Diff"},
        xaxis_tickvals=labels,
        width=800
    )

    line(
        per_layer_logit_diffs,
        hovermode="x unified",
        title=f"Per Layer Logit Difference From Accumulated Residual Stream for {token_pairs_names[i]}",
        labels={"x": "Layer", "y": "Logit Diff"},
        xaxis_tickvals=labels,
        width=800
    )
    model = model.to("cpu")
    linreg_cache = linreg_cache.to("cpu")
    per_head_residual, labels = linreg_cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
    per_head_residual = per_head_residual.to("cpu")
    per_head_residual = einops.rearrange(
        per_head_residual,
        "(layer head) ... -> layer head ...",
        layer=model.cfg.n_layers
    )
    per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, linreg_cache)

    imshow(
        per_head_logit_diffs,
        labels={"x":"Head", "y":"Layer"},
        title="Logit Difference From Each Head",
        width=600
    )
    # # Save the logit lens plot
    # fig = line(
    #     logit_lens_logit_diffs,
    #     hovermode="x unified", 
    #     title=f"Logit Difference From Accumulated Residual Stream for {token_pairs_names[i]}",
    #     labels={"x": "Layer", "y": "Logit Diff"},
    #     xaxis_tickvals=labels,
    #     width=800,
    #     return_fig=True
    # )
    # # Save the file, creating it if it doesn't exist
    # output_path = f"../docs/logit_lens_{token_pairs_names[i].replace(' ', '_')}.html"
    # with open(output_path, 'w') as f:
    #     fig.write_html(f)
        # Create the plot
    # fig = create_logit_lens_plot(
    #     logit_lens_logit_diffs,
    #     labels,
    #     token_pairs_names[i]
    # )
    
    # # Save just the plot HTML
    # output_path = f"../docs/logit_lens_{token_pairs_names[i].replace(' ', '_')}.html"
    # with open(output_path, 'w') as f:
    #     f.write(fig.to_html(
    #         full_html=False,
    #         include_plotlyjs='cdn',
    #         config={'displayModeBar': False}
    #     ))
    # plot_html = create_logit_lens_plot(
    #     logit_lens_logit_diffs,
    #     labels,
    #     token_pairs_names[i]
    # )
    
    # # Save just the plot HTML
    # output_path = f"../docs/logit_lens_{token_pairs_names[i].replace(' ', '_')}.html"
    # with open(output_path, 'w') as f:
    #     f.write(plot_html)

INFO:__main__:Number of comparisons: 21
INFO:__main__:Number of token_pairs: 21
INFO:__main__:Processing comparison 0: gold vs linear_regression
DEBUG:__main__:Per prompt logit difference for comparison 'gold vs linear_regression': tensor([-0.2383], dtype=torch.float16)
DEBUG:__main__:Average logit difference for comparison 'gold vs linear_regression': -0.23828125
DEBUG:__main__:Final residual stream shape: torch.Size([1, 1154, 4096])
DEBUG:__main__:Answer residual directions shape: torch.Size([1, 2, 4096])
DEBUG:__main__:Logit difference directions shape: torch.Size([1, 4096])
DEBUG:__main__:Scaled residual stream shape: torch.Size([1, 4096])
DEBUG:__main__:Logit diff directions shape: torch.Size([1, 4096])
DEBUG:__main__:Scaled residual stream shape: torch.Size([65, 1, 4096])
DEBUG:__main__:Logit diff directions shape: torch.Size([1, 4096])
DEBUG:__main__:Scaled residual stream shape: torch.Size([65, 1, 4096])
DEBUG:__main__:Logit diff directions shape: torch.Size([1, 4096])


Token pair shape: torch.Size([1, 2])


Moving model to device:  cpu
Tried to stack head results when they weren't cached. Computing head results now


: 

In [11]:
from optim_hunter.model_utils import get_numerical_tokens

numerical_tokens = get_numerical_tokens(model)

model.clear_contexts()
linreg_tokens = linreg_tokens.to('cpu')
linreg_logits = linreg_logits.to('cpu')
linreg_cache = linreg_cache.to('cpu')

# Extract predictions across all datasets and convert to tensors
predictions_store = {
    predictor: t.tensor([
        dataset["predictions"][predictor] 
        for dataset in linreg_data_store
    ], dtype=t.float32)
    for predictor in linreg_data_store[0]["predictions"].keys()
}

# Print predictions as strings
print("\nPredictions:")
print("-" * 50)
for predictor_name, predictor_tensor in predictions_store.items():
    print(f"\n{predictor_name}:")
    for i, pred in enumerate(predictor_tensor):
        print(f"  Sample {i}: {pred:.2f}")


# Now predictions_store is a dict where each value is a tensor of shape [batch]
# Example:
# {
#     'gold': tensor([952.17, ...], dtype=torch.float32),
#     'linear_regression': tensor([936.12, ...], dtype=torch.float32),
#     'knn': tensor([555.85, ...], dtype=torch.float32),
#     ...
# }

# Verify the shape and contents
for predictor_name, predictor_tensor in predictions_store.items():
    print(f"\nPredictor: {predictor_name}")
    print(f"Shape: {predictor_tensor.shape}")
    print(f"Values: {predictor_tensor}")

    # Get the residual directions for numerical tokens
    numerical_token_ids = t.tensor(list(numerical_tokens.values()), device="cpu").expand(predictor_tensor.shape[0], -1)

    print(f"Numerical token IDs shape: {numerical_token_ids.shape}")

    def logits_to_numeric_mse(
        logits: Float[Tensor, "batch seq d_vocab"],
        predicted_values: Float[Tensor, "batch"],
        numerical_tokens: dict,
        per_prompt: bool = False
    ) -> Float[Tensor, "*batch"]:
        '''
        Returns weighted MSE between the logit predictions for numeric tokens and correct values.
        
        Args:
            logits: Model logits with shape [batch, seq, d_vocab]
            predicted_values: Tensor of correct numeric values for each batch
            numerical_tokens: Dictionary mapping string digits to token IDs
            per_prompt: If True, return array of MSE per prompt, else return mean
            
        Returns:
            If per_prompt=True: Tensor of MSE values per prompt
            If per_prompt=False: Mean MSE across all prompts
        '''
        # Extract logits for final token
        final_logits = logits[:, -1, :]  # Shape: [batch, d_vocab]
        
        # Get numeric token IDs and their corresponding values
        numeric_ids = []
        numeric_values = []
        for digit, token_id in numerical_tokens.items():
            try:
                value = float(digit)
                numeric_ids.append(token_id)
                numeric_values.append(value)
            except ValueError:
                continue
                
        numeric_ids = t.tensor(numeric_ids, device=logits.device)
        numeric_values = t.tensor(numeric_values, device=logits.device)
        
        # Get probabilities for numeric tokens
        numeric_logits = final_logits[:, numeric_ids]  # Shape: [batch, num_numeric_tokens]
        numeric_probs = t.softmax(numeric_logits, dim=-1)  # Shape: [batch, num_numeric_tokens]
        
        # Calculate expected value for each prompt
        expected_values = (numeric_probs * numeric_values.unsqueeze(0)).sum(dim=-1)  # Shape: [batch]
        
        # Calculate MSE
        mse = (expected_values - predicted_values) ** 2  # Shape: [batch]
        
        if per_prompt:
            return mse
        else:
            return mse.mean()

    original_per_prompt_mse = logits_to_numeric_mse(linreg_logits, predictor_tensor, numerical_tokens, per_prompt=True)
    # print(f"Per prompt numeric MSE for comparison '{token_pairs_names[i]}': {original_per_prompt_mse}")
    original_average_mse = logits_to_numeric_mse(linreg_logits, predictor_tensor, numerical_tokens)
    # print(f"Average numeric MSE for comparison '{token_pairs_names[i]}': {original_average_mse}")

    # Retrieve final residual stream
    final_residual_stream: Float[Tensor, "batch seq d_model"] = linreg_cache["resid_post", -1]
    logger.debug(f"Final residual stream shape: {final_residual_stream.shape}")
    final_token_residual_stream: Float[Tensor, "batch d_model"] = final_residual_stream[:, -1, :]

    # Compute residual directions for numeric MSE
    numeric_residual_directions = model.tokens_to_residual_directions(numerical_token_ids).to("cpu")
    logger.debug(f"Numeric residual directions shape: {numeric_residual_directions.shape}")

    # # Calculate weighted residual directions based on predictor values
    # weighted_residual_directions = numeric_residual_directions * predictor_tensor.unsqueeze(-1)  # [batch d_model]
    # logger.debug(f"Weighted residual directions shape: {weighted_residual_directions.shape}")

    def residual_stack_to_numeric_logits(
        residual_stack: Float[Tensor, "... batch d_model"],
        cache: ActivationCache,
        numeric_tokens: dict,
        numeric_residual_directions: Float[Tensor, "batch num_tokens d_model"]
    ) -> Float[Tensor, "... batch num_tokens"]:
        '''
        Gets the logits for all numeric tokens at each layer of the residual stack.
        
        Args:
            residual_stack: Stack of residual streams
            cache: Model activation cache
            numeric_tokens: Dictionary mapping string digits to token IDs
            
        Returns:
            Tensor of logits for each numeric token at each layer
        '''
        # Apply LayerNorm scaling
        scaled_residual_stream = cache.apply_ln_to_stack(residual_stack, layer=-1, pos_slice=-1)
        scaled_residual_stream = scaled_residual_stream.to("cpu")
        numeric_residual_directions = numeric_residual_directions.to(dtype=scaled_residual_stream.dtype) 
        
        # Project residual stream onto numeric directions
        numeric_logits = einops.einsum(
            scaled_residual_stream, 
            numeric_residual_directions,
            "... batch d_model, batch num_tokens d_model -> ... batch num_tokens"
        )
        
        return numeric_logits

    # # Verify residual stack computation
    # t.testing.assert_close(
    #     residual_stack_to_logit_diff(final_token_residual_stream.to(t.float32), linreg_cache.to(t.float32)),
    #     original_average_logit_diff.to(t.float32),
    #     rtol=5e-3,  # Increased tolerance
    #     atol=5e-3
    # )

    # Get accumulated residuals across layers
    accumulated_residual, labels = linreg_cache.accumulated_resid(
        layer=-1, 
        incl_mid=True, 
        pos_slice=-1, 
        return_labels=True
    )

    # Get logits for numeric tokens at each layer
    numeric_logits = residual_stack_to_numeric_logits(
        accumulated_residual,
        linreg_cache,
        numerical_tokens,
        numeric_residual_directions
    )

    # Print shape of numeric_logits tensor
    print("Shape of numeric_logits:", numeric_logits.shape)

    # Convert logits to probabilities
    numeric_probs = t.softmax(numeric_logits, dim=-1)  # [..., batch, num_tokens]

    print("Shape of numeric_logits:", numeric_probs.shape)
    print("Size of numerical tokens:", len(numerical_tokens))


    # Calculate expected numeric value at each layer
    # Filter out non-numeric tokens and create numeric_values tensor
    numeric_values = []
    for digit in numerical_tokens.keys():
        try:
            # Handle both integer digits and floating point numbers
            if isinstance(digit, (int, float)):
                value = float(digit)
            else:
                value = float(digit.replace(',', ''))  # Handle any commas in number strings
            numeric_values.append(value)
        except ValueError as e:
            print(f"ERROR converting '{digit}' to float:", e)
            continue
    numeric_values = t.tensor(numeric_values, device=numeric_probs.device)

    # Reshape numeric_values for broadcasting
    numeric_values = numeric_values.view(1, 1, -1)  # Shape: [1, 1, num_tokens]

    # Calculate expected values with broadcasting
    expected_values = (numeric_probs * numeric_values).sum(dim=-1)  # [..., batch]

    # Plot how the predicted value evolves across layers
    line(
        expected_values.mean(dim=1),  # Average across batch
        hovermode="x unified",
        title="Evolution of Numeric Prediction Across Layers",
        labels={"x": "Layer", "y": "Predicted Value"},
        xaxis_tickvals=labels,
        width=800
    )

    # Also plot the confidence (entropy) in the prediction
    entropy = -(numeric_probs * t.log(numeric_probs)).sum(dim=-1)  # [..., batch]
    line(
        entropy.mean(dim=1),  # Average across batch
        hovermode="x unified",
        title="Evolution of Prediction Confidence Across Layers",
        labels={"x": "Layer", "y": "Entropy (lower = more confident)"},
        xaxis_tickvals=labels,
        width=800
    )
    # # Save the logit lens plot
    # fig = line(
    #     logit_lens_logit_diffs,
    #     hovermode="x unified", 
    #     title=f"Logit Difference From Accumulated Residual Stream for {token_pairs_names[i]}",
    #     labels={"x": "Layer", "y": "Logit Diff"},
    #     xaxis_tickvals=labels,
    #     width=800,
    #     return_fig=True
    # )
    # # Save the file, creating it if it doesn't exist
    # output_path = f"../docs/logit_lens_{token_pairs_names[i].replace(' ', '_')}.html"
    # with open(output_path, 'w') as f:
    #     fig.write_html(f)
        # Create the plot
    # fig = create_logit_lens_plot(
    #     logit_lens_logit_diffs,
    #     labels,
    #     token_pairs_names[i]
    # )
    
    # # Save just the plot HTML
    # output_path = f"../docs/logit_lens_{token_pairs_names[i].replace(' ', '_')}.html"
    # with open(output_path, 'w') as f:
    #     f.write(fig.to_html(
    #         full_html=False,
    #         include_plotlyjs='cdn',
    #         config={'displayModeBar': False}
    #     ))

DEBUG:__main__:Final residual stream shape: torch.Size([1, 1154, 4096])


DEBUG:__main__:Numeric residual directions shape: torch.Size([1, 1110, 4096])



Predictions:
--------------------------------------------------

gold:
  Sample 0: 952.17

linear_regression:
  Sample 0: 936.12

knn:
  Sample 0: 555.85

random_forest:
  Sample 0: 751.89

average:
  Sample 0: 514.97

last:
  Sample 0: 125.36

random:
  Sample 0: 502.58

Predictor: gold
Shape: torch.Size([1])
Values: tensor([952.1700])
Numerical token IDs shape: torch.Size([1, 1110])
Shape of numeric_logits: torch.Size([65, 1, 1110])
Shape of numeric_logits: torch.Size([65, 1, 1110])
Size of numerical tokens: 1110


DEBUG:__main__:Final residual stream shape: torch.Size([1, 1154, 4096])
DEBUG:__main__:Numeric residual directions shape: torch.Size([1, 1110, 4096])



Predictor: linear_regression
Shape: torch.Size([1])
Values: tensor([936.1222])
Numerical token IDs shape: torch.Size([1, 1110])
Shape of numeric_logits: torch.Size([65, 1, 1110])
Shape of numeric_logits: torch.Size([65, 1, 1110])
Size of numerical tokens: 1110


...

Shape of numeric_logits: torch.Size([65, 1, 1110])
Shape of numeric_logits: torch.Size([65, 1, 1110])
Size of numerical tokens: 1110


DEBUG:__main__:Final residual stream shape: torch.Size([1, 1154, 4096])



Predictor: random
Shape: torch.Size([1])
Values: tensor([502.5800])
Numerical token IDs shape: torch.Size([1, 1110])


DEBUG:__main__:Numeric residual directions shape: torch.Size([1, 1110, 4096])


Shape of numeric_logits: torch.Size([65, 1, 1110])
Shape of numeric_logits: torch.Size([65, 1, 1110])
Size of numerical tokens: 1110


In [17]:
from optim_hunter.model_utils import get_numerical_tokens

numerical_tokens = get_numerical_tokens(model)

model.clear_contexts()
linreg_tokens = linreg_tokens.to('cpu')
linreg_logits = linreg_logits.to('cpu')
linreg_cache = linreg_cache.to('cpu')

# Extract predictions across all datasets and convert to tensors
predictions_store = {
    predictor: t.tensor([
        dataset["predictions"][predictor] 
        for dataset in linreg_data_store
    ], dtype=t.float32)
    for predictor in linreg_data_store[0]["predictions"].keys()
}

# Print predictions as strings
print("\nPredictions:")
print("-" * 50)
for predictor_name, predictor_tensor in predictions_store.items():
    print(f"\n{predictor_name}:")
    for i, pred in enumerate(predictor_tensor):
        print(f"  Sample {i}: {pred:.2f}")


# Now predictions_store is a dict where each value is a tensor of shape [batch]
# Example:
# {
#     'gold': tensor([952.17, ...], dtype=torch.float32),
#     'linear_regression': tensor([936.12, ...], dtype=torch.float32),
#     'knn': tensor([555.85, ...], dtype=torch.float32),
#     ...
# }

# Verify the shape and contents
for predictor_name, predictor_tensor in predictions_store.items():
    print(f"\nPredictor: {predictor_name}")
    print(f"Shape: {predictor_tensor.shape}")
    print(f"Values: {predictor_tensor}")

    # Get the residual directions for numerical tokens
    numerical_token_ids = t.tensor(list(numerical_tokens.values()), device="cpu").expand(predictor_tensor.shape[0], -1)

    print(f"Numerical token IDs shape: {numerical_token_ids.shape}")

    def logits_to_numeric_mse(
        logits: Float[Tensor, "batch seq d_vocab"],
        predicted_values: Float[Tensor, "batch"],
        numerical_tokens: dict,
        per_prompt: bool = False
    ) -> Float[Tensor, "*batch"]:
        '''
        Returns weighted MSE between the logit predictions for numeric tokens and correct values.
        
        Args:
            logits: Model logits with shape [batch, seq, d_vocab]
            predicted_values: Tensor of correct numeric values for each batch
            numerical_tokens: Dictionary mapping string digits to token IDs
            per_prompt: If True, return array of MSE per prompt, else return mean
            
        Returns:
            If per_prompt=True: Tensor of MSE values per prompt
            If per_prompt=False: Mean MSE across all prompts
        '''
        # Extract logits for final token
        final_logits = logits[:, -1, :]  # Shape: [batch, d_vocab]
        
        # Get numeric token IDs and their corresponding values
        numeric_ids = []
        numeric_values = []
        for digit, token_id in numerical_tokens.items():
            try:
                value = float(digit)
                numeric_ids.append(token_id)
                numeric_values.append(value)
            except ValueError:
                continue
                
        numeric_ids = t.tensor(numeric_ids, device=logits.device)
        numeric_values = t.tensor(numeric_values, device=logits.device)
        
        # Get probabilities for numeric tokens
        numeric_logits = final_logits[:, numeric_ids]  # Shape: [batch, num_numeric_tokens]
        numeric_probs = t.softmax(numeric_logits, dim=-1)  # Shape: [batch, num_numeric_tokens]
        
        # Calculate expected value for each prompt
        expected_values = (numeric_probs * numeric_values.unsqueeze(0)).sum(dim=-1)  # Shape: [batch]
        
        # Calculate MSE
        mse = (expected_values - predicted_values) ** 2  # Shape: [batch]
        
        if per_prompt:
            return mse
        else:
            return mse.mean()

    original_per_prompt_mse = logits_to_numeric_mse(linreg_logits, predictor_tensor, numerical_tokens, per_prompt=True)
    # print(f"Per prompt numeric MSE for comparison '{token_pairs_names[i]}': {original_per_prompt_mse}")
    original_average_mse = logits_to_numeric_mse(linreg_logits, predictor_tensor, numerical_tokens)
    # print(f"Average numeric MSE for comparison '{token_pairs_names[i]}': {original_average_mse}")

    # Retrieve final residual stream
    final_residual_stream: Float[Tensor, "batch seq d_model"] = linreg_cache["resid_post", -1]
    logger.debug(f"Final residual stream shape: {final_residual_stream.shape}")
    final_token_residual_stream: Float[Tensor, "batch d_model"] = final_residual_stream[:, -1, :]

    # Compute residual directions for numeric MSE
    numeric_residual_directions = model.tokens_to_residual_directions(numerical_token_ids).to("cpu")
    logger.debug(f"Numeric residual directions shape: {numeric_residual_directions.shape}")

    # # Calculate weighted residual directions based on predictor values
    # weighted_residual_directions = numeric_residual_directions * predictor_tensor.unsqueeze(-1)  # [batch d_model]
    # logger.debug(f"Weighted residual directions shape: {weighted_residual_directions.shape}")

    def residual_stack_to_numeric_logits(
        residual_stack: Float[Tensor, "... batch d_model"],
        cache: ActivationCache,
        numeric_tokens: dict,
        numeric_residual_directions: Float[Tensor, "batch num_tokens d_model"]
    ) -> Float[Tensor, "... batch num_tokens"]:
        '''
        Gets the logits for all numeric tokens at each layer of the residual stack.
        
        Args:
            residual_stack: Stack of residual streams
            cache: Model activation cache
            numeric_tokens: Dictionary mapping string digits to token IDs
            
        Returns:
            Tensor of logits for each numeric token at each layer
        '''
        # Apply LayerNorm scaling
        scaled_residual_stream = cache.apply_ln_to_stack(residual_stack, layer=-1, pos_slice=-1)
        scaled_residual_stream = scaled_residual_stream.to("cpu")
        numeric_residual_directions = numeric_residual_directions.to(dtype=scaled_residual_stream.dtype) 
        
        # Project residual stream onto numeric directions
        numeric_logits = einops.einsum(
            scaled_residual_stream, 
            numeric_residual_directions,
            "... batch d_model, batch num_tokens d_model -> ... batch num_tokens"
        )
        
        return numeric_logits
    ###############################################################################
    # Within your for predictor_name, predictor_tensor in predictions_store.items():
    ###############################################################################

    # 1) Retrieve accumulated residuals for all layers (including mid-layer if desired)
    accumulated_residual, labels = linreg_cache.accumulated_resid(
        layer=-1, 
        incl_mid=True,      # Keep mid-layer positions if you want finer granularity
        pos_slice=-1,       # Only final token in each prompt
        return_labels=True
    )
    # accumulated_residual has shape [num_layers_plus_mid, batch, d_model]

    # 2) Project the residual at each layer onto numeric token directions
    #    We'll define a helper function below called get_numeric_logits_per_layer
    def get_numeric_logits_per_layer(
        residual_stack: Float[Tensor, "layers batch d_model"],
        cache: ActivationCache,
        numeric_residual_directions: Float[Tensor, "batch num_tokens d_model"]
    ) -> Float[Tensor, "layers batch num_tokens"]:
        """
        For each layer in residual_stack, apply the final LN to get the correct scaling,
        then dot with numeric_residual_directions to get numeric logits.
        
        Args:
            residual_stack: shape [layers, batch, d_model]
            cache: ActivationCache to apply LN
            numeric_residual_directions: shape [batch, num_tokens, d_model]
            
        Returns:
            numeric_logits: shape [layers, batch, num_tokens]
        """
        # Apply LN to each layer's [batch, d_model]
        # apply_ln_to_stack expects shape [layers, batch, d_model],
        # returns the same shape after LN.
        scaled_residual_stream = cache.apply_ln_to_stack(residual_stack, layer=-1, pos_slice=-1)
        scaled_residual_stream = scaled_residual_stream.to("cpu")

        # Make sure directions have the same dtype
        numeric_residual_directions = numeric_residual_directions.to(dtype=scaled_residual_stream.dtype)

        # Dot product to get logits
        # scaled_residual_stream: [layers, batch, d_model]
        # numeric_residual_directions: [batch, num_tokens, d_model]
        # We'll broadcast batch dimension. We'll need einops or explicit broadcasting:
        numeric_logits = einops.einsum(
            scaled_residual_stream,             # [layers, batch, d_model]
            numeric_residual_directions,        # [batch, num_tokens, d_model]
            "layer b d, b t d -> layer b t"
        )
        return numeric_logits

    numeric_logits_per_layer = get_numeric_logits_per_layer(
        accumulated_residual,
        linreg_cache,
        numeric_residual_directions
    )
    # numeric_logits_per_layer: [num_layers_plus_mid, batch, num_tokens]
    print("Shape of numeric_logits_per_layer:", numeric_logits_per_layer.shape)

    # 3) Convert to probabilities per layer
    numeric_probs_per_layer = t.softmax(numeric_logits_per_layer, dim=-1)  # [layers, batch, num_tokens]
    print("Shape of numeric_probs_per_layer:", numeric_probs_per_layer.shape)

    # 4) Build numeric_values Tensor from your dictionary keys
    numeric_values_list = []
    for digit in numerical_tokens.keys():
        try:
            numeric_values_list.append(float(digit))
        except ValueError:
            pass  # skip if it can't convert
    numeric_values_tensor = t.tensor(numeric_values_list, device=numeric_probs_per_layer.device)

    # 5) Compute expected numeric value per layer: E[number]
    # numeric_probs_per_layer: [layers, batch, num_tokens]
    # numeric_values_tensor: [num_tokens]
    # We'll reshape numeric_values_tensor to broadcast
    numeric_values_reshaped = numeric_values_tensor.view(1, 1, -1)  # [1, 1, num_tokens]

    # Multiply + sum over num_tokens axis
    # We'll get [layers, batch]
    expected_values_per_layer = (numeric_probs_per_layer * numeric_values_reshaped).sum(dim=-1)

    # 6) Compute MSE at each layer. predictor_tensor has shape [batch], the "true" or gold values.
    # We'll broadcast or expand to [layers, batch] so we can do (pred - gold)^2
    # If you want to treat predictor_tensor as the "gold," do so:
    gold_values = predictor_tensor  # shape [batch]
    gold_values_expanded = gold_values.unsqueeze(0)  # shape [1, batch]

    layerwise_mse = (expected_values_per_layer - gold_values_expanded) ** 2  # [layers, batch]
    # Average across the batch dimension
    layerwise_mse_mean = layerwise_mse.mean(dim=-1)  # [layers]

    print(f"Layerwise MSE shape: {layerwise_mse_mean.shape}")

    # 7) Plot how the MSE evolves across layers
    line(
        layerwise_mse_mean,  # shape [layers]
        hovermode="x unified",
        title=f"MSE vs. Gold Across Layers for {predictor_name}",
        labels={"x": "Layer", "y": "MSE"},
        xaxis_tickvals=labels,  # `labels` from accumulated_resid
        width=800
    )

    #########################################################################
    # (Optional) Also plot how the expected numeric value changes across layers
    #########################################################################

    avg_expected_across_batch = expected_values_per_layer.mean(dim=-1)  # [layers]
    line(
        avg_expected_across_batch,
        hovermode="x unified",
        title=f"Expected Numeric Prediction Across Layers for {predictor_name}",
        labels={"x": "Layer", "y": "Predicted Value"},
        xaxis_tickvals=labels,
        width=800
    )

DEBUG:__main__:Final residual stream shape: torch.Size([1, 1154, 4096])
DEBUG:__main__:Numeric residual directions shape: torch.Size([1, 1110, 4096])



Predictions:
--------------------------------------------------

gold:
  Sample 0: 952.17

linear_regression:
  Sample 0: 936.12

knn:
  Sample 0: 555.85

random_forest:
  Sample 0: 751.89

average:
  Sample 0: 514.97

last:
  Sample 0: 125.36

random:
  Sample 0: 502.58

Predictor: gold
Shape: torch.Size([1])
Values: tensor([952.1700])
Numerical token IDs shape: torch.Size([1, 1110])
Shape of numeric_logits_per_layer: torch.Size([65, 1, 1110])
Shape of numeric_probs_per_layer: torch.Size([65, 1, 1110])
Layerwise MSE shape: torch.Size([65])


DEBUG:__main__:Final residual stream shape: torch.Size([1, 1154, 4096])
DEBUG:__main__:Numeric residual directions shape: torch.Size([1, 1110, 4096])



Predictor: linear_regression
Shape: torch.Size([1])
Values: tensor([936.1222])
Numerical token IDs shape: torch.Size([1, 1110])
Shape of numeric_logits_per_layer: torch.Size([65, 1, 1110])
Shape of numeric_probs_per_layer: torch.Size([65, 1, 1110])
Layerwise MSE shape: torch.Size([65])


DEBUG:__main__:Final residual stream shape: torch.Size([1, 1154, 4096])
DEBUG:__main__:Numeric residual directions shape: torch.Size([1, 1110, 4096])


...

Shape of numeric_logits_per_layer: torch.Size([65, 1, 1110])
Shape of numeric_probs_per_layer: torch.Size([65, 1, 1110])
Layerwise MSE shape: torch.Size([65])


DEBUG:__main__:Final residual stream shape: torch.Size([1, 1154, 4096])



Predictor: random
Shape: torch.Size([1])
Values: tensor([502.5800])
Numerical token IDs shape: torch.Size([1, 1110])


DEBUG:__main__:Numeric residual directions shape: torch.Size([1, 1110, 4096])


Shape of numeric_logits_per_layer: torch.Size([65, 1, 1110])
Shape of numeric_probs_per_layer: torch.Size([65, 1, 1110])
Layerwise MSE shape: torch.Size([65])
